## Attention 신경망 구현 및 학습

In [1]:
import time
import random
import tensorflow as tf
from konlpy.tag import Okt #pip install konlpy

## 하이퍼 파라미터

In [2]:
EPOCHS = 100 # 초기 설정값은 200이었으나 시간이 많이 소요되어서 100으로 조정함
NUM_WORDS = 2000

## Encoder

In [3]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        
        # 입력 단어를 embeding 처리함
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        
        # 입력 단어를 LSTM으로 처리함
        # return_sequences=True => 모든 sequence를 돌려줌
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        # H는 모든 hidden state
        H, h, c = self.lstm(x)
        return H, h, c

## Decoder

In [4]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.att = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        # x : shifted output, s0 : 처음 들어 오는 값, c0 : 처음 들어 오는 값
        # H : 인코드에서 나온 모든 hidden 값
        x, s0, c0, H = inputs
        x = self.emb(x)
        
        # S : 모든 hidden state 값
        S, h, c = self.lstm(x, initial_state=[s0, c0])
        
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        
        # Attention value (Query, Key, Value)
        A = self.att([S_, H])
        
        # 모든 Hidden state와 Attention value를 구성
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

## Seq2seq

In [5]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos # start of sentence
        self.eos = eos # end of sentence

    def call(self, inputs, training=False, mask=None):
        if training is True:
            # x : input, y : output
            x, y = inputs
            
            # h : hidden state, c : cell state
            H, h, c = self.enc(x)
            
            # y : output (전체 문장)
            y, _, _ = self.dec((y, h, c, H))
            return y
        else:
            # x : input
            x = inputs
            
            # h : hidden state, c : cell state            
            H, h, c = self.enc(x)
            
            # sos를 tensor type으로 변경
            y = tf.convert_to_tensor(self.sos)
            
            # sos의 shape을 (1) 구조에서 (1,1) 구조로 변경
            y = tf.reshape(y, (1, 1))

            # 64개 단어로 구성된 문장을 tensor type로 생성
            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c, H])
                
                # tf.argmax(y, axis=-1)는 정답을 추출
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [6]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:] # SOS 를 넣기 위한 구조
    shifted_labels = labels[:, :-1] # EOS를 넣기 위한 구조
    
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [7]:
#java JDK 설치 필요
dataset_file = '../dataset/chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    # morphs는 형태소 분석 기능
    seq = [' '.join(okt.morphs(line)) for line in lines] 

# 입력데이타는 질문 문장과 답변 문장의 반복으로 구성되어 있음
print("원본문장 =>",lines[:10])

# 형태소 분석이 완료된 문장을 두칸씩 이동해서 데이타 가져오기
questions = seq[::2] 
print("\n가공된 질문 문장 =>",questions[:10])

# Tab글자는 Start of Sentence임
answers = ['\t ' + lines for lines in seq[1::2]] 
print("\n가공된 답변 문장 =>",answers[:10])

num_sample = len(questions)

perm = list(range(num_sample))
print("\nperm =>",perm[:10])

random.seed(0)
random.shuffle(perm) # 리스트 값을 섞음
print("\nperm =>",perm[:10])

train_q = list()
train_a = list()
test_q = list()
test_a = list()

# train 데이타와 test 데이타를 분리함
for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:  # 5의 몫을 구함. 500//5 -> 100
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)
        
# NUM_WORDS개의 단어만 토큰으로 저장하고 필터에 있는 문자는 제외하는 설정
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')
# 실제로 문장을 받아서 토큰 처리함
tokenizer.fit_on_texts(train_q + train_a)

# 문장들을 토큰화된 숫자로 치환시킴
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)
print('\n질문 문장=>',train_q[:10])
print('\n질문 토큰 문장=>',train_q_seq[:10])
print('\n아메리카노 토큰 =>',tokenizer.word_index['아메리카노']) # 중요한 함수


test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

# 문장의 길이를 64개 단어로 구성하도록하며, 문장이 짧을 경우 빈칸을 채워준다.
# 출력 문장은 입력 문장에 EOS를 추가하기 위하여 한단어가 더 길게 구성됨
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)
print('\n질문 패딩 문장=>',x_train[:10])
print('\n답변 패딩 문장=>',y_train[:10])

x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

원본문장 => ['아이스아메리카노 하나요\n', '테이크아웃하실 건가요?\n', '저 카푸치노로 주문할게요\n', '시럽은 얼마나 뿌려드릴까요?\n', '저 도장 다 모았는데 나중에 써도 되나요?\n', '네 다음에 써도 됩니다\n', '이 기프티콘 여기서 사용할 수 있나요?\n', '사용 가능하십니다\n', '자바칩 프라푸치노에 휘핑 빼고요\n', '6600원입니다\n']

가공된 질문 문장 => ['아이스 아메리카노 하나요 \n', '저 카푸치노 로 주문 할게요 \n', '저 도장 다 모았는데 나중 에 써도 되나요 ? \n', '이 기프티콘 여기 서 사용 할 수 있나요 ? \n', '자바 칩 프라푸치노 에 휘핑 빼고요 \n', '여기 기프티콘 대면 되죠 ? \n', '" 따뜻한 아메리카노 한 잔 , 아이스 라떼 한 잔이요 ." \n', '" 네 , 그런데 중간 에 테이크 아웃 도 가능한가요 ?" \n', '" 네 , 스콘 도 3 개 같이 주세요 ." \n', '카푸치노 차갑게 되나요 ? \n']

가공된 답변 문장 => ['\t 테이크아웃 하실 건가 요 ? \n', '\t 시럽 은 얼마나 뿌려 드릴 까요 ? \n', '\t 네 다음 에 써도 됩니다 \n', '\t 사용 가능하십니다 \n', '\t 6600원 입니다 \n', '\t 네 현금영수증 해드릴까 요 ? \n', '\t 드시고 가시나요 ? \n', '\t " 네 , 카운터 로 오시 면 테이크 아웃 잔 에 담아 드려요 ." \n', '\t 스콘 도 드시고 가시나요 ? \n', '\t 가능합니다 \n']

perm => [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

perm => [419, 459, 130, 431, 370, 26, 201, 56, 366, 108]

질문 문장=> ['아이스 아메리카노 하나요 \n', '저 카푸치노 로 주문 할게요 \n', '저 도장 다 모았는데 나중 에 써도 되나요 ? \n', '이 기프티콘 여기 서 사용 할 수 있나요 ? \n', '자바 칩 프라푸치노 에 

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [8]:
# Create model
# 문장의 시작 토큰과 문장의 종료 토큰을 설정함
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [9]:
# 시간측정 필요. 노트북의 경우 1 epoch당 30초 정도 소요됨
t = time.time()
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()
    ct = time.time()
    print('{} seconds elapsed.'.format(ct - t))
    t = ct       

Epoch 1, Loss: 2.872729778289795, Accuracy: 83.12577819824219
47.24568510055542 seconds elapsed.
Epoch 2, Loss: 0.6804295778274536, Accuracy: 90.3900375366211
27.35051465034485 seconds elapsed.
Epoch 3, Loss: 0.5826170444488525, Accuracy: 90.65241241455078
27.272101879119873 seconds elapsed.
Epoch 4, Loss: 0.5663137435913086, Accuracy: 91.05967712402344
27.64638590812683 seconds elapsed.
Epoch 5, Loss: 0.5492140054702759, Accuracy: 91.16932678222656
27.615357875823975 seconds elapsed.
Epoch 6, Loss: 0.537538468837738, Accuracy: 91.17716217041016
27.731791734695435 seconds elapsed.
Epoch 7, Loss: 0.520825982093811, Accuracy: 91.16541290283203
27.143136739730835 seconds elapsed.
Epoch 8, Loss: 0.5313218832015991, Accuracy: 91.23198699951172
26.398317098617554 seconds elapsed.
Epoch 9, Loss: 0.5190854072570801, Accuracy: 91.23198699951172
28.0290367603302 seconds elapsed.
Epoch 10, Loss: 0.49917763471603394, Accuracy: 91.37296295166016
27.892261028289795 seconds elapsed.
Epoch 11, Loss: 0

Epoch 84, Loss: 0.09395474940538406, Accuracy: 98.01457214355469
28.147758960723877 seconds elapsed.
Epoch 85, Loss: 0.09000364691019058, Accuracy: 98.2025375366211
27.924251317977905 seconds elapsed.
Epoch 86, Loss: 0.08669168502092361, Accuracy: 98.27693939208984
27.74576449394226 seconds elapsed.
Epoch 87, Loss: 0.0829743817448616, Accuracy: 98.36309814453125
28.165634870529175 seconds elapsed.
Epoch 88, Loss: 0.07889547944068909, Accuracy: 98.46099853515625
28.01335859298706 seconds elapsed.
Epoch 89, Loss: 0.07541748881340027, Accuracy: 98.54714965820312
27.858492612838745 seconds elapsed.
Epoch 90, Loss: 0.07086700946092606, Accuracy: 98.60197448730469
27.976922035217285 seconds elapsed.
Epoch 91, Loss: 0.06849238276481628, Accuracy: 98.75469970703125
27.99857449531555 seconds elapsed.
Epoch 92, Loss: 0.06606905907392502, Accuracy: 98.80952453613281
28.247173070907593 seconds elapsed.
Epoch 93, Loss: 0.06238634139299393, Accuracy: 98.90350341796875
28.725528717041016 seconds elap

## 테스트 루프

In [10]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('질문 : ', test_text)
    print('답변 : ', gt_text)
    print('예측 : ', texts)

_
질문 :  ['여기 기프티콘 되죠 \n']
답변 :  ['\t 네 현금영수증 해드릴까 요 \n']
예측 :  ['네 진동 벨 울리면 찾으러 오세요 \n']
_
질문 :  ['네 에 테이크 아웃 도 가능한가요 \n']
답변 :  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
예측 :  ['아뇨 현재 법적 으로 금지 \n']
_
질문 :  ['아메리카노 톨 사이즈 로 주세요 \n']
답변 :  ['\t 따뜻한 거 로 드릴 까요 \n']
예측 :  ['아이스 아메리카노 한잔 주문 도 와 드리겠습니다 \n']
_
질문 :  ['진동 을 따로 주시나요 \n']
답변 :  ['\t 주 번호 로 드리겠습니다 \n']
예측 :  ['네 포인트 할인 가능하세요 \n']
_
질문 :  ['자리 있나요 \n']
답변 :  ['\t 네 있습니다 \n']
예측 :  ['네 유효 차는 있습니다 \n']
_
질문 :  ['그럼 루이보스 밀크 티 하나 \n']
답변 :  ['\t 네 알겠습니다 \n']
예측 :  ['네 4500원 입니다 \n']
_
질문 :  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
답변 :  ['\t 네 \n']
예측 :  ['네 알겠습니다 \n']
_
질문 :  ['아메리카노 한 잔 에 얼마 죠 \n']
답변 :  ['\t 입니다 \n']
예측 :  ['4000원 입니다 \n']
_
질문 :  ['얼마나 \n']
답변 :  ['\t 바로 만들어 드릴게요 \n']
예측 :  ['따뜻한 걸 로 드릴 까요 \n']
_
질문 :  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
답변 :  ['\t 네 더 없으세요 \n']
예측 :  ['네 저희 꺼 4500원 입니다 \n']
_
질문 :  ['아메리카노 는 어떤 종류 가 있나요 \n']
답변 :  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
예측 :  ['네 고객 님 사이즈 는 다 되었습니다 \n']
_
질문 :  ['카카오 페이 로 결제 가능한가요 \n']


_
질문 :  ['티라미수 는 있나요 \n']
답변 :  ['\t 네 티라미수 는 있습니다 \n']
예측 :  ['네 그럼요 \n']
_
질문 :  ['네 현금영수증 해주세요 \n']
답변 :  ['\t 네 드시고 가시나요 \n']
예측 :  ['네 번호 찍어주세요 \n']
_
질문 :  ['샷 추가 해주세요 \n']
답변 :  ['\t 네 알겠습니다 \n']
예측 :  ['네 앞 에 도 와 드리겠습니다 \n']
_
질문 :  ['얼마 에요 \n']
답변 :  ['\t 만 원 입니다 \n']
예측 :  ['베이글 과 동일하게 2000원 입니다 \n']
_
질문 :  ['아이스 아메리카노 랑 샌드위치 주세요 \n']
답변 :  ['\t 10시 에 세트 할인 가능하세요 \n']
예측 :  ['아이스 아메리카노 안 안 입니다 \n']
